# Дерево решений на размеченном без обучения датасете

In [1]:
from genetic import optimal_canny_for_ds as genetic_canny_for_ds
from data_wrappers import BDD10kEdges, CityScapesEdges, apply_roi
from bayesian_canny import optimal_canny_for_ds as bayesian_canny_for_ds
from metrics import NormalizedFoM, Jaccard
from torch.utils.data import Dataset
from canny import canny
import pandas as pd
from benchmark import binarize
import typing as tp
from numpy import ndarray, mean
from rich.progress import track

In [2]:
class ROI(Dataset):
    def __init__(self, proxy: Dataset):
        super().__init__()
        self.proxy = proxy
    def __getitem__(self, index):
        img, edge = self.proxy[index]
        return apply_roi(img), apply_roi(edge)
    def __len__(self):
        return len(self.proxy)

## Разметка датасета

In [9]:
# С помощью генетического алгоритма
bdd10k = BDD10kEdges("./data/bdd100k", "train")
bdd10k = ROI(bdd10k)
metric = Jaccard(0.2) # Используем Jaccard потому что Normalized FoM считается относительно долго
optimal_canny_params = genetic_canny_for_ds(bdd10k, metric)
optimal_canny_params.to_csv("genetic_bdd10k_canny_params.csv")

Output()

In [11]:
cityscapes = CityScapesEdges("./data/cityscapes", "train")
cityscapes = ROI(cityscapes)
metric = Jaccard(0.5) # Используем Jaccard потому что Normalized FoM считается относительно долго
optimal_canny_params = genetic_canny_for_ds(cityscapes, metric)
optimal_canny_params.to_csv("genetic_cityscapes_canny_params.csv")

Output()

In [31]:
# Проверка качества
def check_suitability(param_df: pd.DataFrame, ds: Dataset, metric: tp.Callable[[ndarray, ndarray], float]):
    mv = []
    for (img, edge), (i, params) in track(zip(ds, param_df.iterrows()), "Checking suitability"):
        thr1, thr2 = params["thr1"], params["thr2"]
        prediction = canny(img, thr1, thr2)
        prediction = binarize(prediction, 250)
        edge = binarize(edge, 250)
        mv.append(metric(prediction, edge))
    return mean(mv)

check_suitability(
    pd.read_csv("genetic_bdd10k_canny_params.csv", index_col=0),
    ROI(BDD10kEdges("./data/bdd100k", "train")),
    NormalizedFoM(0.2, 0.2)
)

Output()

0.49390189463255113

In [32]:
check_suitability(
    pd.read_csv("genetic_cityscapes_canny_params.csv"),
    ROI(CityScapesEdges("./data/cityscapes", "train")),
    Jaccard(0.5)
)

Output()

0.08141240199400909

## Генерация признаков

In [34]:
from feature_generation import generate_features

In [35]:
ds = ROI(BDD10kEdges("./data/bdd100k", "train"))
generate_features(ds, "bdd100k_features.csv")

Output()

In [36]:
ds = ROI(CityScapesEdges("./data/cityscapes", "train"))
generate_features(ds, "cityscapes_features.csv")

Output()

## Обучение

In [3]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor

In [4]:
X_cityscapes = pd.read_csv("cityscapes_features.csv", index_col=0)
y_cityscapes = pd.read_csv("genetic_cityscapes_canny_params.csv", index_col=0)

X_bdd10k = pd.read_csv("bdd100k_features.csv", index_col=0)
y_bdd10k = pd.read_csv("genetic_bdd10k_canny_params.csv", index_col=0)

In [5]:
cityscapes = pd.merge(X_cityscapes, y_cityscapes, left_index=True, right_index=True)
bdd10k = pd.merge(X_bdd10k, y_bdd10k, left_index=True, right_index=True)
data = pd.concat([cityscapes, bdd10k], ignore_index=True)

In [6]:
data = data.sample(frac=1)
X, y = data.drop(["thr1", "thr2"], axis=1), data[["thr1", "thr2"]]

In [27]:
regressor = DecisionTreeRegressor()

parameters = {
    "max_depth": [3, 5, 7, 10],
    "min_samples_leaf": [1, 10, 20, 40, 70],
#     "criterion": ["squared_error", "friedman_mse", "absolute_error", "poisson"]
}

grid_search = GridSearchCV(regressor, parameters, verbose=3)
grid_search.fit(X, y)

grid_search.best_params_

Fitting 5 folds for each of 20 candidates, totalling 100 fits
[CV 1/5] END ...max_depth=3, min_samples_leaf=1;, score=0.365 total time=   0.1s
[CV 2/5] END ...max_depth=3, min_samples_leaf=1;, score=0.375 total time=   0.1s
[CV 3/5] END ...max_depth=3, min_samples_leaf=1;, score=0.386 total time=   0.1s
[CV 4/5] END ...max_depth=3, min_samples_leaf=1;, score=0.382 total time=   0.1s
[CV 5/5] END ...max_depth=3, min_samples_leaf=1;, score=0.419 total time=   0.1s
[CV 1/5] END ..max_depth=3, min_samples_leaf=10;, score=0.365 total time=   0.1s
[CV 2/5] END ..max_depth=3, min_samples_leaf=10;, score=0.375 total time=   0.1s
[CV 3/5] END ..max_depth=3, min_samples_leaf=10;, score=0.386 total time=   0.1s
[CV 4/5] END ..max_depth=3, min_samples_leaf=10;, score=0.382 total time=   0.1s
[CV 5/5] END ..max_depth=3, min_samples_leaf=10;, score=0.419 total time=   0.1s
[CV 1/5] END ..max_depth=3, min_samples_leaf=20;, score=0.365 total time=   0.1s
[CV 2/5] END ..max_depth=3, min_samples_leaf=20

{'max_depth': 7, 'min_samples_leaf': 70}

In [7]:
regressor = DecisionTreeRegressor(max_depth=7, min_samples_leaf=70)
regressor.fit(X, y)

DecisionTreeRegressor(max_depth=7, min_samples_leaf=70)

## Тестирование

In [13]:
import numpy as np
from feature_generation import gray_level_hist, jpeg_compression_vector
from canny import canny

In [30]:
def generate_feature_vector(img):
    return pd.DataFrame(np.concatenate([gray_level_hist(img), jpeg_compression_vector(img)]).reshape(1, -1), columns=X.columns)

In [18]:
def run_benchmark(method: tp.Callable[[np.ndarray], np.ndarray],
                  metric: tp.Callable[[np.ndarray, np.ndarray], float],
                  dataset: Dataset):
    v = []
    for img, gt in track(dataset, "Running benchmark", total=len(dataset)):
        prediction = method(img)
        gt = binarize(gt, 250)
        prediction = binarize(prediction, 250)
        v.append(metric(prediction, gt))
    return np.mean(v)

In [32]:
def tree_method(image: np.ndarray) -> np.ndarray:
    img_v = generate_feature_vector(image)
    prediction = regressor.predict(img_v)
    thr1, thr2 = prediction.tolist()[0]
    return canny(image, thr1, thr2)

In [34]:
run_benchmark(
    tree_method,
    Jaccard(0.5),
    ROI(CityScapesEdges("./data/cityscapes", "val"))
)

Output()

0.08191130375642584

In [35]:
run_benchmark(
    tree_method,
    NormalizedFoM(0.2, 0.2),
    ROI(BDD10kEdges("./data/bdd100k", "val"))
)

Output()

0.4872284086957628